In [1]:
import ee
import geemap
import pandas as pd

# Authenticate and Initialize
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

/opt/homebrew/Caskroom/miniconda/base/envs/edsml-rs/lib/python3.10/site-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [25]:
# Create an interactive map centered on Nagasaki
Map = geemap.Map(center=[32.75, 129.87], zoom=12)

# If you have the geojson, you can load it:
# nagasaki_admin = geemap.geojson_to_ee('path/to/your/nagasaki.geojson')

# For now, let's use a point to filter the collection
nagasaki_pt = ee.Geometry.Point([129.87, 32.75])
Map.addLayer(nagasaki_pt, {'color': 'red'}, 'Nagasaki Center')
Map

Map(center=[32.75, 129.87], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…

In [26]:
# Filter the global database for Nagasaki Prefecture
# Level 1 is usually provinces/prefectures
studyArea = ee.FeatureCollection("FAO/GAUL/2015/level1") \
            .filter(ee.Filter.eq('ADM1_NAME', 'Nagasaki'))

Map.centerObject(studyArea, 10)
Map.addLayer(studyArea, {'color': 'blue', 'fillColor': '00000000'}, 'Nagasaki Prefecture')

In [ ]:
def preprocess_landsat(image):
    # Optical bands scaling
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    # Thermal bands scaling (if needed)
    thermal_bands = image.select('ST_B10').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)
    
def get_nagasaki_stats(year):
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    
    # Load and Preprocess
    dataset = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
               .filterBounds(nagasaki_pt)
               .filterDate(start_date, end_date)
               .map(preprocess_landsat) # Apply scaling here
               .sort('CLOUD_COVER')
               .first()
               .clip(studyArea))
    
    # Calculate NDVI: (NIR - Red) / (NIR + Red)
    ndvi = dataset.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    
    return dataset, ndvi

# Re-run and visualize
img_year, ndvi_year = get_nagasaki_stats(2025)

# Use a more detailed palette to see the contrast
ndvi_vis = {
    'min': 0, 
    'max': 0.8, 
    'palette': ['#FFFFFF', '#CE7E45', '#DF923D', '#F1B555', '#FCD163', '#99B718', '#74A901', '#66A200', '#529400', '#3E8601', '#207401', '#056201', '#004C00', '#023B01', '#012E01', '#011D01', '#011301']
}
Map.addLayer(ndvi_year, ndvi_vis, 'NDVI Corrected')

In [28]:
def get_nagasaki_indices(year):
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    
    # Load and scale
    img = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
           .filterBounds(nagasaki_pt)
           .filterDate(start_date, end_date)
           .map(preprocess_landsat)
           .sort('CLOUD_COVER')
           .first()
           .clip(studyArea))
    
    # Calculate Indices
    ndvi = img.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    ndwi = img.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')
    ndbi = img.normalizedDifference(['SR_B6', 'SR_B5']).rename('NDBI')
    
    # Add indices back to the image as new bands
    return img.addBands([ndvi, ndwi, ndbi])

# Run for 2023
full_img = get_nagasaki_indices(2023)

In [29]:
# 1. NDWI - Water (Blue palette)
ndwi_vis = {'min': -0.5, 'max': 0.5, 'palette': ['white', 'blue']}
Map.addLayer(full_img.select('NDWI'), ndwi_vis, 'Water Index (NDWI)')

# 2. NDBI - Urban (Red/Grey palette)
# Note: NDBI can be noisy; built-up areas usually have values > 0
ndbi_vis = {'min': -0.5, 'max': 0.5, 'palette': ['white', 'red']}
Map.addLayer(full_img.select('NDBI'), ndbi_vis, 'Built-up Index (NDBI)')

In [22]:
# 1. Load the SRTM Data
dem = ee.Image("USGS/SRTMGL1_003")

# 2. Clip to your studyArea
# Ensure 'studyArea' is defined from your geojson/GAUL step
nagasaki_dem = dem.clip(studyArea)

# 3. Define visualization parameters
# We use a 'terrain' palette: Lowlands (greens) to Highlands (browns/whites)
dem_vis = {
    'min': 0,
    'max': 600,
    'palette': ['#006600', '#002200', '#fff5d7', '#d5b980', '#ecd2a5', '#ebebeb', '#f5f5f5']
}

# 4. Add to map
Map = geemap.Map()
Map.centerObject(studyArea, 11)
Map.addLayer(nagasaki_dem, dem_vis, 'Elevation (m)')
Map

Map(center=[33.22035729137407, 129.6200058865179], controls=(WidgetControl(options=['position', 'transparent_b…